-Constraint to set one node to 1 for the ending node: $(1-\sum_i y_i + \sum_{i\neq j} 2y_iy_j)$

-Constraint to set other nodes to 0 or 2: $(\sum_jy_j)(2-\sum_jy_j)^2= \sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq j}y_iy_jy_k$. Adding ancilla variables to make it QUBO:
$\sum_i y_i-2\sum_{i\neq j}y_iy_j+6\sum_{i\neq j,i\neq k,j\neq k}[w_{i,j}y_k]+6\sum_{i\neq j,i\neq k,j\neq k}[y_iy_j-2w_{i,j}(y_i+y_j)+3w_{i,j}]$

-Constraint to set one node to $n$ for the starting node: $(\sum_iy_i-n)^2=n^2-(2n-1)\sum_i y_i+\sum_{i\neq j} 2y_iy_j$

-Minimum distance objective function:  $w\sum_{i \notin start, i \notin start} y_i$ if $w$ is not there 0 would be a viable solution too. While the constraints only add a penalty or reward of 1/-1, the objective function's penalty grows linearly with the routs length. Therefore, it is imperetive that we nulify the effect of this growth with a small weight. A weight too big wont do shit and a weight too small will make the adiabatic gap very tiny and inceases the error rate. 

In [ ]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import FixedEmbeddingComposite
import dwave_networkx as dnx
import hybrid
import dimod
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import sys
from pathlib import Path
pp = str(Path('.').absolute().parent)
if pp not in sys.path:
    sys.path.append(pp)

In [ ]:
#import the necessary shit
from refactor.essentials import (
    RectGridGraph, create_qubo, SA, optimize_qannealer,
    is_this_an_answer,
)
from qpr.quantum_utils import find_embedding_minorminer

In [ ]:
#prepare the netlist

net_start=[(0,0)]
net_end=[(0,2)]

G = RectGridGraph(3, 3)

In [ ]:
%%time
params = {'weight_objective': 0.3594062134771152,
 'weight_end': 1.1453614963677057,
 'weight_start': 1.6267212231734953,
 'weight_others': 0.4445755049371818,
 'weight_and': 6.323717667375186}
Q=create_qubo(G, net_start, net_end, params)
dwave_sampler = DWaveSampler(solver={'lower_noise': True, 'qpu': True})
A = dwave_sampler.edgelist
embedding, chain_len = find_embedding_minorminer(Q, A, num_tries=10)
## the shortest chain_len I've seen with num_tries=1000 is 5
## (SP: takes 2.5 mins on my machine, SAS: 1:08 on mine)
display(chain_len)

In [ ]:
connectivity_structure = dnx.chimera_graph(16,16)
fig=plt.figure(figsize=(25, 25))
dnx.draw_chimera_embedding(connectivity_structure, embedding)

# QPU

In [ ]:
fixed_sampler = FixedEmbeddingComposite(
            DWaveSampler(solver={'lower_noise': True, 'qpu': True}), embedding
            )
q_response = optimize_qannealer(fixed_sampler, Q, params={'chain_strength': 20, 'annealing_time': 99, 'num_reads': 10000})
display(q_response.first)
best_q_answer = q_response.first.sample

In [ ]:
G.draw()

edge_set = G.qubo_answer2node_pairs(q_response.samples()[0])
G.highlight_edge_list(edge_set)

In [ ]:
is_this_an_answer(q_response.samples()[0], G, net_start, net_end)

In [ ]:
def make_ax_grid(n, ax_h=4, ax_w=6, ncols=4):
    nrows = int(np.ceil(n / ncols))
    fig_h = nrows * ax_h
    fig_w = ncols * ax_w
    return plt.subplots(nrows=nrows, ncols=ncols, figsize=(fig_w, fig_h))

# Exact solver

In [ ]:
%%time
exact_response = dimod.ExactSolver().sample_qubo(Q)
display(exact_response.record)

In [ ]:
# .data() sorts by energy by defaults but returns an iterator (not a SampleSet)
# the iterator yields a named tuple
# .samples(n) sort by energy, take at most n samples, return a SampleArray
# which is a view, mapping the var names to the values (i.e returns dicts), It is
# indexable i.e. .samples()[10] works
# .record returns record array of Sample objects which is basically a 
# numpy-scliceable list of named tuples (samples). Also .record.energy
# returns a numpy array of energies, .record.samples returns a 2d numpy
# array of qubo answers etc.
# Iterating over the SampleSet, calls .samples() internally, i.e. it gets sorted
# .first calls data() internally so it does the sorting anyway!

# This function returns all the min energy solutions as a list of {var name: val} dicts
def get_all_min_energy(sample_set):
    min_energy = np.min(sample_set.record.energy)
    # use .record since it is slicing friendly, this returns a 2d-array-like recarray
    records = sample_set.record[sample_set.record.energy == min_energy]
    # make dicts out of each answer using the original var names (i.e. sample_set.variables)
    return [dict(zip(sample_set.variables, i.sample)) for i in records], min_energy

In [ ]:
def plot_all_exact_solutions(min_energy_sols):
    fig, axes = make_ax_grid(len(min_energy_sols))
    display(len(min_energy_sols))
    
    for ax, answer_dict in zip(axes.flat, min_energy_sols):
        G.draw(edge_labs=False, ax=ax)  # edge_labs=False)
    
        edge_set = G.qubo_answer2node_pairs(answer_dict)
        G.highlight_edge_list(edge_set, ax=ax)
    fig.tight_layout()

In [ ]:
min_energy_sols, _ = get_all_min_energy(exact_response)
plot_all_exact_solutions(min_energy_sols)

In [ ]:
def check_against_exact(ans,exact_min_energy_sols):
    #ans is the answer from QPU or hybrid solver. 
    #exact_min_energy_sols is the set of all possible solutions from the exact solver
    return (ans in exact_min_energy_sols)

In [ ]:
print(check_against_exact(q_response.samples()[0],min_energy_sols))

# hybrid solution

In [ ]:
# Construct a problem
offset=0.0
#vartype = dimod.BINARY
bqm = dimod.BinaryQuadraticModel.from_qubo(Q, offset)

# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

# Solve the problem
init_state = hybrid.State.from_problem(bqm)
final_state = workflow.run(init_state).result()

# Print results
print("Solution: sample={.samples.first}".format(final_state))

In [ ]:
G.draw(edge_labs=False)  # edge_labs=False)

edge_set = G.qubo_answer2node_pairs(final_state.samples.first[0])
G.highlight_edge_list(edge_set)


# simulated  annealing

In order to optimize the parameters of QUBO (and later QPU) we employ s simulated annealing algorithm. 

In [ ]:
a=SA(4, params={'weight_objective': [1, 0, 2, 0, 0.1], 'weight_end': [1, 0, 2, 0, 0.1],
                                           'weight_start': [1, 0, 2, 0, 0.1] ,'weight_others': [1, 0, 2, 0, 0.1],
                                           'weight_and': [6, 0, 30, 0, 0.1],
                                           'chain_strength': [7, 3, 800, 0, 0.1], 'annealing_time': [500, 99, 1000, 1, 10]
                                          },
                 T=1, T_min=0.01, alpha=0.8, max_iter=5)

In [ ]:
a.anneal()

In [ ]:
a.costs

In [ ]:
A[-100:-1]